In [ ]:
import sys
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

pd.set_option("display.max_rows", None)

from argparse import ArgumentParser
import itertools as it

print("adding code to the pythonpath...")
code = "/home/anna/Documents/code/python"
if code not in sys.path:
    sys.path.insert(1, code)
import pipeline
from pipeline.utils import validate

maybe plot w log scale

In [ ]:
df_dict = {}

for prot in ["tyk2", "mcl1", "p38"]:
    file = f"/home/anna/Documents/benchmark/extracted/{prot}/perturbing_overlap.dat"
    df = pd.read_csv(file)
    df["score"] = np.nan

    perturbations, ligs = pipeline.analysis.get_info_network(
        f"/home/anna/Documents/benchmark/{prot}_benchmark/execution_model/network_lomap.dat"
    )

    score_dict = {}
    with open(
        f"/home/anna/Documents/benchmark/{prot}_benchmark/execution_model/network_lomap_scores.dat"
    ) as lfile:
        for line in lfile:
            score_dict[
                f"{line.split(',')[0].strip()}~{line.split(',')[1].strip()}"
            ] = float(line.split(",")[-1].strip())

    for index, row in df.iterrows():
        if row["perturbation"] not in perturbations:
            df = df.drop(index)
        else:
            df.at[index, "score"] = score_dict[row["perturbation"]]

    df_dict[prot] = df

frames = []
for key in df_dict.keys():
    frames.append(df_dict[key])

all_df = pd.concat(frames)
df = all_df
# df = df_dict["p38"]

In [ ]:
# histogram of failed run in terms of perturbing atoms

df_has = df[df["percen_overlap_okay"] >= 0]
df_none = (
    pd.merge(df_has, df, how="outer", indicator=True)
    .query("_merge != 'both'")
    .drop("_merge", axis=1)
    .reset_index(drop=True)
)
print(len(df_none))
df_none["perturbing_atoms"].plot.hist(bins=10)
plt.xlabel("perturbing atoms")

In [ ]:
df_plot = df.dropna()

df_plot.plot.scatter("score", "too_small_avg", c="perturbing_atoms", colormap="viridis")

df_plot.plot.scatter(
    "score", "percen_overlap_okay", c="perturbing_atoms", colormap="viridis"
)

In [ ]:
df_plot = df.dropna()

df_plot.plot.scatter(
    "perturbing_atoms", "percen_overlap_okay", c="too_small_avg", colormap="viridis"
)
# plt.title(f"")
# plt.xlabel("no. of perturbing atoms")
# plt.ylabel("percentage of okay overlap")

In [ ]:
df_plot.plot.scatter(
    "perturbing_atoms", "too_small_avg", c="percen_overlap_okay", colormap="viridis"
)
# plt.title(f"")
# plt.xlabel("no. of perturbing atoms")
# plt.ylabel("average no. of too small off-diagonals per leg")

In [ ]:
df_plot.plot.scatter(
    "percen_overlap_okay", "too_small_avg", c="perturbing_atoms", colormap="viridis"
)

In [ ]:
# exclude outliers as needed
df3 = df_plot[df_plot["diff_to_exp"] >= 20]
df_out = (
    pd.merge(df3, df_plot, how="outer", indicator=True)
    .query("_merge != 'both'")
    .drop("_merge", axis=1)
    .reset_index(drop=True)
)

# df_out = df_plot

df_out.plot.scatter(
    "perturbing_atoms", "diff_to_exp", c="percen_overlap_okay", colormap="viridis"
)

In [ ]:
columns = ["perturbing_atoms", "percen_overlap_okay", "too_small_avg"]
bins = [6, 3, 5]
for column, bin in zip(columns, bins):
    fig = plt.figure()
    df_plot = df[column]
    df_plot.plot.hist(subplots=True, bins=bin)
    plt.title(column)

In [ ]:
# for per engine
eng = "GROMACS"
df2 = df[df["engine"] == eng]
df_plot = df2.dropna()
df_plot.plot.scatter(
    "perturbing_atoms", "percen_overlap_okay", c="too_small_avg", colormap="viridis"
)
df_plot.plot.scatter(
    "perturbing_atoms", "too_small_avg", c="percen_overlap_okay", colormap="viridis"
)
df_plot.plot.scatter(
    "percen_overlap_okay", "too_small_avg", c="perturbing_atoms", colormap="viridis"
)

In [ ]:
engs = ["AMBER", "SOMD", "GROMACS"]
eng_dict = {}

col_dict = pipeline.analysis.plotting_engines.set_colours()

for eng in engs:
    df2 = df[df["engine"] == eng]
    eng_dict[eng] = df2

for eng in engs:
    df_plot = eng_dict[eng].dropna()
    ax = df_plot.plot.scatter(
        "perturbing_atoms", "percen_overlap_okay", c=col_dict[eng]
    )

df_plot = eng_dict["AMBER"].dropna()
ax1 = df_plot.plot.scatter(
    "perturbing_atoms", "percen_overlap_okay", c=col_dict["AMBER"]
)
df_plot = eng_dict["SOMD"].dropna()
ax2 = df_plot.plot.scatter(
    "perturbing_atoms", "percen_overlap_okay", c=col_dict["SOMD"], ax=ax1
)
df_plot = eng_dict["GROMACS"].dropna()
ax3 = df_plot.plot.scatter(
    "perturbing_atoms", "percen_overlap_okay", c=col_dict["GROMACS"], ax=ax1
)
plt.legend(col_dict, loc="upper right")
print(ax1 == ax2 == ax3)

In [ ]:
engs = ["AMBER", "SOMD", "GROMACS"]
eng_dict = {}

col_dict = pipeline.analysis.plotting_engines.set_colours()

for eng in engs:
    df2 = df[df["engine"] == eng]
    df3 = df2[df2["diff_to_exp"] >= 5]
    df4 = (
        pd.merge(df3, df2, how="outer", indicator=True)
        .query("_merge != 'both'")
        .drop("_merge", axis=1)
        .reset_index(drop=True)
    )
    eng_dict[eng] = df4

for eng in engs:
    df_plot = eng_dict[eng].dropna()
    ax = df_plot.plot.scatter("perturbing_atoms", "diff_to_exp", c=col_dict[eng])

df_plot = eng_dict["AMBER"].dropna()
ax1 = df_plot.plot.scatter("perturbing_atoms", "diff_to_exp", c=col_dict["AMBER"])
df_plot = eng_dict["SOMD"].dropna()
ax2 = df_plot.plot.scatter(
    "perturbing_atoms", "diff_to_exp", c=col_dict["SOMD"], ax=ax1
)
df_plot = eng_dict["GROMACS"].dropna()
ax3 = df_plot.plot.scatter(
    "perturbing_atoms", "diff_to_exp", c=col_dict["GROMACS"], ax=ax1
)
plt.legend(col_dict, loc="upper right")
print(ax1 == ax2 == ax3)

checking which perts are bad overlap

In [ ]:
file = f"/home/anna/Documents/benchmark/extracted/mcl1/perturbing_overlap.dat"

eng_dict_ok = {"AMBER": None, "SOMD": None, "GROMACS": None}
eng_dict_not = {"AMBER": None, "SOMD": None, "GROMACS": None}

for engine in ["SOMD", "AMBER", "GROMACS"]:
    print(engine)
    perts_okay = []
    perts_not = []
    with open(file, "r") as f:
        for line in f.readlines():
            pert = line.split(",")[0].strip()
            overlap_okay = line.split(",")[3].strip()
            eng = line.split(",")[1].strip()

            if eng == engine:
                if overlap_okay == "100.0":  #  or overlap_okay == "50.0"
                    if pert not in perts_okay:
                        perts_okay.append(pert)
                else:
                    if pert not in perts_not:
                        perts_not.append(pert)

    # for pert in perts_not:
    #     if pert in perts_okay:
    #         perts_okay.remove(pert)

    print(len(perts_okay))
    print(perts_okay)
    print(len(perts_not))
    print(perts_not)
    print(" ")

    eng_dict_ok[engine] = perts_okay
    eng_dict_not[engine] = perts_not


both_perts = []

for pert in eng_dict_ok["AMBER"]:
    if pert in eng_dict_ok["SOMD"]:
        both_perts.append(pert)

print(len(both_perts))
print(both_perts)
print(" ")

all_perts = eng_dict_ok["SOMD"] + eng_dict_not["SOMD"]
not_okay = []
for pert in all_perts:
    if pert not in both_perts:
        not_okay.append(pert)

print(len(not_okay))
print(not_okay)
print(" ")

In [ ]:
prot = pipeline_protocol()
prot.num_lambda(16)
pert_dict = {(pert.split("~")[0], pert.split("~")[1]): "None" for pert in not_okay}
write_network(pert_dict, prot, "new_mcl1_network_both2.dat")